In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./category.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_manufacturer = df['Manufacturer'].tolist()
list_manufacturer_number = df['Manufacturer_No.'].tolist()
list_category = df['Category'].tolist()
list_category_code = df['Category_Code'].tolist()
list_category_number = df['Category_No.'].tolist()

print('总数量：' + str(len(list_category)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for manufacturer, manufacturer_number, category, category_code, category_number in zip(list_manufacturer, list_manufacturer_number, list_category, list_category_code, list_category_number):
    work.put_nowait((manufacturer, int(manufacturer_number), category, category_code, int(category_number)))

def crawler():
    global df
    
    while not work.empty():
        manufacturer, manufacturer_number, category, category_code, category_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                data = 'func=partsearchgetparttypessdd&mfr=' + manufacturer.replace('+', '%2B').replace(' ', '+').replace('&', '%26').replace('/', '%2F').replace(',', '%2C').replace('Ö', '%C3%96').replace('Ü', '%C3%9C') + '&pg=' + category.replace(' ', '+').replace('&', '%26').replace('/', '%2F') + '&id=partsearch_007&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_replace_sections']['ddrepl[parttype_partsearch_007][container]']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_type = html.xpath('//select[@id="parttype_partsearch_007"]/option/text()')
                
                # = = = = = = = = = = = = = = =
                
                list_type_code = html.xpath('//select[@id="parttype_partsearch_007"]/option/@value')

                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com/en/parts/'+manufacturer+','+type_+','+type_code for type_, type_code in zip(list_type[1:], list_type_code[1:])]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Manufacturer_No.': manufacturer_number,
                                        'Manufacturer': manufacturer,
                                        'Category_No.': category_number,
                                        'Category': category,
                                        'Category_Code': category_code,
                                        'Type_No.': [_+1 for _ in range(len(list_type)-1)],
                                        'Type': list_type[1:],
                                        'Type_Code': list_type_code[1:],
                                        'Url': list_url})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer_No.': manufacturer_number,
                                     'Manufacturer': manufacturer,
                                     'Category_No.': category_number,
                                     'Category': category,
                                     'Category_Code': category_code}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(manufacturer + ', ' + category +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./type_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：2454

3M, Body & Lamp Assembly  <->  [ok] - 剩余数量：2449
3M, Engine  <->  [ok] - 剩余数量：2448
3M, Garage Equipment  <->  [ok] - 剩余数量：2447
3M, Interior  <->  [ok] - 剩余数量：2446
3M, Fuel & Air  <->  [ok] - 剩余数量：2445
AAE, Steering  <->  [ok] - 剩余数量：2444
ACC, Interior  <->  [ok] - 剩余数量：2443
ACCEL, Fuel & Air  <->  [ok] - 剩余数量：2442
ACCEL, Ignition  <->  [ok] - 剩余数量：2441
ACDELCO, Belt Drive  <->  [ok] - 剩余数量：2440
3D, Interior  <->  [ok] - 剩余数量：2439
ACDELCO, Brake & Wheel Hub  <->  [ok] - 剩余数量：2438
ACDELCO, Cooling System  <->  [ok] - 剩余数量：2437
ACDELCO, Electrical  <->  [ok] - 剩余数量：2436
ACDELCO, Drivetrain  <->  [ok] - 剩余数量：2435
ACDELCO, Electrical-Bulb & Socket  <->  [ok] - 剩余数量：2434
3D, Body & Lamp Assembly  <->  [ok] - 剩余数量：2433
ACDELCO, Electrical-Switch & Relay  <->  [ok] - 剩余数量：2432
ACDELCO, Exhaust & Emission  <->  [ok] - 剩余数量：2431
ACDELCO, Fuel & Air  <->  [ok] - 剩余数量：2430
ACDELCO, Engine  <->  [ok] - 剩余数量：2429
ACDELCO, Heat & Air Conditioning  <->  [ok] - 剩余数量：2428
ACDELCO, Hardware  <->

ATP, Apparel & Gifts  <->  [ok] - 剩余数量：2273
ATP, Brake & Wheel Hub  <->  [ok] - 剩余数量：2272
ATP, Drivetrain  <->  [ok] - 剩余数量：2271
ATP, Electrical  <->  [ok] - 剩余数量：2270
ATP, Electrical-Switch & Relay  <->  [ok] - 剩余数量：2269
ATP, Engine  <->  [ok] - 剩余数量：2268
ATP, Fuel & Air  <->  [ok] - 剩余数量：2267
ATP, Electrical-Connector  <->  [ok] - 剩余数量：2266
ATP, Exhaust & Emission  <->  [ok] - 剩余数量：2265
ATP, Heat & Air Conditioning  <->  [ok] - 剩余数量：2264
ATP, Interior  <->  [ok] - 剩余数量：2263
ATP, Hardware  <->  [ok] - 剩余数量：2262
ATP, Transmission-Manual  <->  [ok] - 剩余数量：2261
ATP, Steering  <->  [ok] - 剩余数量：2260
ATSCO, Steering  <->  [ok] - 剩余数量：2259
AUS INJECTION INC., Fuel & Air  <->  [ok] - 剩余数量：2258
AUTO 7, Body & Lamp Assembly  <->  [ok] - 剩余数量：2257
AUTO 7, Belt Drive  <->  [ok] - 剩余数量：2256
AUTO 7, Brake & Wheel Hub  <->  [ok] - 剩余数量：2255
ATP, Transmission-Automatic  <->  [ok] - 剩余数量：2254
AUTO 7, Cooling System  <->  [ok] - 剩余数量：2253
AUTO 7, Drivetrain  <->  [ok] - 剩余数量：2252
AUTO 7, Electrical  <-

BELDEN, Electrical  <->  [ok] - 剩余数量：2104
BENDIX, Brake & Wheel Hub  <->  [ok] - 剩余数量：2103
BELDEN, Ignition  <->  [ok] - 剩余数量：2102
BENDIX, Transmission-Manual  <->  [ok] - 剩余数量：2101
BENDIX, Wheel  <->  [ok] - 剩余数量：2100
BH SENS, Wheel  <->  [ok] - 剩余数量：2099
BILSTEIN, Engine  <->  [ok] - 剩余数量：2098
BENTLEY PUBLISHERS, Literature  <->  [ok] - 剩余数量：2097
BILSTEIN, Steering  <->  [ok] - 剩余数量：2096
BETTER BRAKE PARTS, Brake & Wheel Hub  <->  [ok] - 剩余数量：2095
BILSTEIN, Suspension  <->  [ok] - 剩余数量：2094
BOGE, Suspension  <->  [ok] - 剩余数量：2093
BOOMERANG, Body & Lamp Assembly  <->  [ok] - 剩余数量：2092
BOGE, Steering  <->  [ok] - 剩余数量：2091
BORG-WARNER, Electrical  <->  [ok] - 剩余数量：2090
BORG-WARNER, Engine  <->  [ok] - 剩余数量：2089
BORG-WARNER, Exhaust & Emission  <->  [ok] - 剩余数量：2088
BORG-WARNER, Electrical-Bulb & Socket  <->  [ok] - 剩余数量：2087
BORG-WARNER, Electrical-Switch & Relay  <->  [ok] - 剩余数量：2086
BORG-WARNER, Fuel & Air  <->  [ok] - 剩余数量：2085
BORG-WARNER, Ignition  <->  [ok] - 剩余数量：2084
BORG-WARN

CONTINENTAL, Fuel & Air  <->  [ok] - 剩余数量：1936
CONTINENTAL, Hoses/Lines & Clamps  <->  [ok] - 剩余数量：1935
CONTINENTAL, Steering  <->  [ok] - 剩余数量：1934
CONTINENTAL, Transmission-Automatic  <->  [ok] - 剩余数量：1933
CORTECO, Cooling System  <->  [ok] - 剩余数量：1932
CORTECO, Drivetrain  <->  [ok] - 剩余数量：1931
CORTECO, Electrical  <->  [ok] - 剩余数量：1930
CORTECO, Exhaust & Emission  <->  [ok] - 剩余数量：1929
CORTECO, Fuel & Air  <->  [ok] - 剩余数量：1928
CONTINENTAL, Wiper & Washer  <->  [ok] - 剩余数量：1927
CORTECO, Engine  <->  [ok] - 剩余数量：1926
CORTECO, Steering  <->  [ok] - 剩余数量：1925
CORTECO, Ignition  <->  [ok] - 剩余数量：1924
CORTECO, Transmission-Manual  <->  [ok] - 剩余数量：1923
CROWN REMANUFACTURING, Ignition  <->  [ok] - 剩余数量：1922
CORTECO, Transmission-Automatic  <->  [ok] - 剩余数量：1921
COVERLAY, Interior  <->  [ok] - 剩余数量：1920
CROWN REMANUFACTURING, Steering  <->  [ok] - 剩余数量：1919
CROWN REMANUFACTURING, Wiper & Washer  <->  [ok] - 剩余数量：1918
CRP INDUSTRIES, Engine  <->  [ok] - 剩余数量：1917
CSF, Cooling System  <->  [

ELRING, Transmission-Automatic  <->  [ok] - 剩余数量：1760
EMPIRE, Drivetrain  <->  [ok] - 剩余数量：1759
EMPIRE, Steering  <->  [ok] - 剩余数量：1758
ENERGY SUSPENSION, Body & Lamp Assembly  <->  [ok] - 剩余数量：1757
ENERGY SUSPENSION, Drivetrain  <->  [ok] - 剩余数量：1756
ELRING, Hardware  <->  [ok] - 剩余数量：1755
ENERGY SUSPENSION, Garage Equipment  <->  [ok] - 剩余数量：1754
ENERGY SUSPENSION, Engine  <->  [ok] - 剩余数量：1753
ENERGY SUSPENSION, Cooling System  <->  [ok] - 剩余数量：1752
ENERGY SUSPENSION, Suspension  <->  [ok] - 剩余数量：1751
ENERGY SUSPENSION, Steering  <->  [ok] - 剩余数量：1750
ELRING, Transmission-Manual  <->  [ok] - 剩余数量：1749
ENERGY SUSPENSION, Hardware  <->  [ok] - 剩余数量：1748
ENERGY SUSPENSION, Transmission-Manual  <->  [ok] - 剩余数量：1747
ENERGY SUSPENSION, Transmission-Automatic  <->  [ok] - 剩余数量：1746
ENERGY SUSPENSION, Wheel  <->  [ok] - 剩余数量：1745
ENGINE DYNAMICS, Engine  <->  [ok] - 剩余数量：1744
ENGINETECH, Exhaust & Emission  <->  [ok] - 剩余数量：1743
ENGINETECH, Engine  <->  [ok] - 剩余数量：1742
ENGINETECH, Fuel & 

...

PROFESSIONAL PARTS SWEDEN, Heat & Air Conditioning  <->  [ok] - 剩余数量：732
PROFESSIONAL PARTS SWEDEN, Ignition  <->  [ok] - 剩余数量：731
PROFESSIONAL PARTS SWEDEN, Hoses/Lines & Clamps  <->  [ok] - 剩余数量：730
PROFESSIONAL PARTS SWEDEN, Interior  <->  [ok] - 剩余数量：729
PROFESSIONAL PARTS SWEDEN, Steering  <->  [ok] - 剩余数量：728
PROFESSIONAL PARTS SWEDEN, Suspension  <->  [ok] - 剩余数量：727
PROFESSIONAL PARTS SWEDEN, Transmission-Automatic  <->  [ok] - 剩余数量：726
PROFESSIONAL PARTS SWEDEN, Transmission-Manual  <->  [ok] - 剩余数量：725
PROFESSIONAL PARTS SWEDEN, Wheel  <->  [ok] - 剩余数量：724
PROFESSIONAL'S CHOICE, Fuel & Air  <->  [ok] - 剩余数量：723
PROFORCE, Suspension  <->  [ok] - 剩余数量：722
PROFESSIONAL PARTS SWEDEN, Wiper & Washer  <->  [ok] - 剩余数量：721
PROFORCE, Brake & Wheel Hub  <->  [ok] - 剩余数量：720
PROMASTER BRAKE SYSTEMS, Brake & Wheel Hub  <->  [ok] - 剩余数量：719
PROMOTIVE FILTERS, Fuel & Air  <->  [ok] - 剩余数量：718
PRONTO, Fuel & Air  <->  [ok] - 剩余数量：717
PRONTO, Heat & Air Conditioning  <->  [ok] - 剩余数量：716
PR

SHEE-MAR, Brake & Wheel Hub  <->  [ok] - 剩余数量：561
SHEE-MAR, Body & Lamp Assembly  <->  [ok] - 剩余数量：560
SHEE-MAR, Cooling System  <->  [ok] - 剩余数量：559
SHEE-MAR, Electrical-Switch & Relay  <->  [ok] - 剩余数量：558
SHEE-MAR, Drivetrain  <->  [ok] - 剩余数量：557
SHEE-MAR, Electrical-Bulb & Socket  <->  [ok] - 剩余数量：556
SHEE-MAR, Engine  <->  [ok] - 剩余数量：555
SHEE-MAR, Exhaust & Emission  <->  [ok] - 剩余数量：554
SHEE-MAR, Heat & Air Conditioning  <->  [ok] - 剩余数量：553
SHEE-MAR, Fuel & Air  <->  [ok] - 剩余数量：552
SHEE-MAR, Ignition  <->  [ok] - 剩余数量：551
SHEE-MAR, Interior  <->  [ok] - 剩余数量：550
SHEE-MAR, Steering  <->  [ok] - 剩余数量：549
SHEE-MAR, Suspension  <->  [ok] - 剩余数量：548
SHEE-MAR, Transmission-Automatic  <->  [ok] - 剩余数量：547
SHEE-MAR, Electrical-Connector  <->  [ok] - 剩余数量：546
SHEE-MAR, Electrical  <->  [ok] - 剩余数量：545
SILLA COOLING SYSTEM, Heat & Air Conditioning  <->  [ok] - 剩余数量：544
SILV-O-LITE, Engine  <->  [ok] - 剩余数量：543
SHEE-MAR, Transmission-Manual  <->  [ok] - 剩余数量：542
SKF, Body & Lamp Assembl

TEKONSHA, Electrical-Switch & Relay  <->  [ok] - 剩余数量：390
TIMKEN, Drivetrain  <->  [ok] - 剩余数量：389
TIMKEN, Electrical  <->  [ok] - 剩余数量：388
TIMKEN, Electrical-Connector  <->  [ok] - 剩余数量：387
TIMKEN, Engine  <->  [ok] - 剩余数量：386
TIMKEN, Heat & Air Conditioning  <->  [ok] - 剩余数量：385
TIMKEN, Steering  <->  [ok] - 剩余数量：384
TIMKEN, Suspension  <->  [ok] - 剩余数量：383
TIMKEN, Transmission-Automatic  <->  [ok] - 剩余数量：382
TIMKEN, Transmission-Manual  <->  [ok] - 剩余数量：381
TOMCO, INC., Exhaust & Emission  <->  [ok] - 剩余数量：380
TOMCO, INC., Fuel & Air  <->  [ok] - 剩余数量：379
TOYOTA, Belt Drive  <->  [ok] - 剩余数量：378
TOYOTA, Body & Lamp Assembly  <->  [ok] - 剩余数量：377
TOYOTA, Brake & Wheel Hub  <->  [ok] - 剩余数量：376
TOYOTA, Cooling System  <->  [ok] - 剩余数量：375
TOYOTA, Drivetrain  <->  [ok] - 剩余数量：374
TOYOTA, Electrical  <->  [ok] - 剩余数量：373
TOYOTA, Electrical-Connector  <->  [ok] - 剩余数量：372
TOYOTA, Electrical-Bulb & Socket  <->  [ok] - 剩余数量：371
TOYOTA, Engine  <->  [ok] - 剩余数量：370
TOYOTA, Electrical-Switch

VALEO, Body & Lamp Assembly  <->  [ok] - 剩余数量：220
VALEO, Drivetrain  <->  [ok] - 剩余数量：219
VAICO, Engine  <->  [ok] - 剩余数量：218
VALEO, Brake & Wheel Hub  <->  [ok] - 剩余数量：217
VALEO, Cooling System  <->  [ok] - 剩余数量：216
VALEO, Engine  <->  [ok] - 剩余数量：215
VALEO, Electrical  <->  [ok] - 剩余数量：214
VALEO, Electrical-Switch & Relay  <->  [ok] - 剩余数量：213
VALEO, Ignition  <->  [ok] - 剩余数量：212
VALEO, Interior  <->  [ok] - 剩余数量：211
VALEO, Transmission-Manual  <->  [ok] - 剩余数量：210
VALEO, Heat & Air Conditioning  <->  [ok] - 剩余数量：209
VALEO, Wiper & Washer  <->  [ok] - 剩余数量：208
VALVOLINE, Brake & Wheel Hub  <->  [ok] - 剩余数量：207
VALVOLINE, Cooling System  <->  [ok] - 剩余数量：206
VALVOLINE, Drivetrain  <->  [ok] - 剩余数量：205
VALVOLINE, Exhaust & Emission  <->  [ok] - 剩余数量：204
VALVOLINE, Fuel & Air  <->  [ok] - 剩余数量：203
VALEO, Exhaust & Emission  <->  [ok] - 剩余数量：202
VALVOLINE, Engine  <->  [ok] - 剩余数量：201
VALVOLINE, Garage Equipment  <->  [ok] - 剩余数量：200
VALVOLINE, Steering  <->  [ok] - 剩余数量：199
VALVOLINE, 

WESTIN, Interior  <->  [ok] - 剩余数量：47
WESTLING, Wiper & Washer  <->  [ok] - 剩余数量：46
WESTLING, Steering  <->  [ok] - 剩余数量：45
WIX, Cooling System  <->  [ok] - 剩余数量：44
WIX, Engine  <->  [ok] - 剩余数量：43
WHITE KNIGHT, Wheel  <->  [ok] - 剩余数量：42
WIX, Fuel & Air  <->  [ok] - 剩余数量：41
WIX, Heat & Air Conditioning  <->  [ok] - 剩余数量：40
WIX, Steering  <->  [ok] - 剩余数量：39
WIX, Transmission-Automatic  <->  [ok] - 剩余数量：38
WJB, Brake & Wheel Hub  <->  [ok] - 剩余数量：37
WIX, Exhaust & Emission  <->  [ok] - 剩余数量：36
WJB, Cooling System  <->  [ok] - 剩余数量：35
WJB, Electrical  <->  [ok] - 剩余数量：34
WJB, Engine  <->  [ok] - 剩余数量：33
WJB, Ignition  <->  [ok] - 剩余数量：32
WJB, Heat & Air Conditioning  <->  [ok] - 剩余数量：31
WJB, Transmission-Automatic  <->  [ok] - 剩余数量：30
WJB, Steering  <->  [ok] - 剩余数量：29
WJB, Transmission-Manual  <->  [ok] - 剩余数量：28
WOLVERINE, Engine  <->  [ok] - 剩余数量：27
WORLD BRAKE, Brake & Wheel Hub  <->  [ok] - 剩余数量：26
WVE, Body & Lamp Assembly  <->  [ok] - 剩余数量：25
WVE, Brake & Wheel Hub  <->  [ok] - 剩